On va tenter de simuler une pseudo carte scolaire. En effet, notre modélisation du lien entre taux de réussite au baccalauréat et revenu souffre probablement du fait qu'on ait associé le revenu des élèves à celui des communes de leur lycée. 
Or, des élèves viennent d'autres communes non pourvues d'établissement scolaire et le revenu médian peut être très hétérogène entre deux communes. 
On va donc essayer de refaire une carte scolaire de la manière suivante :
- Chaque commune sera associée à un point géographique. Si elle est dépourvue de lycée, on l'associera au lycée le plus proche. 
- On ponderera alors un revenu médian pour chaque lycée en fonction du revenu médian de chaque commune qui lui est associée et de la population des communes prises en compte. On posera donc l'hypothèse que la proportion de bacheliers potentiels est identique dans chaque commune (ce qui est discutable étant donné qu'il y'a une ségrégation spatiale générationnelle sur le territoire français)
- On procédera ainsi à une nouvelle régression avec les données obtenues

On doit donc créer une base de données adaptée. Il faut qu'on ait dedans
- Chaque établissement scolaire, avec son nombre de candidats et le taux de réussite par filières. 
- Chaque commune avec le revenu médian associé
- Les coordonnées géographiques des lycées et des communes 
En plus des bases de données utilisées précédemment, nous avons utilisé une base de données de La Poste pour avoir la longitude et la latitude de chaque commune. 

In [142]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download
import seaborn as sns

  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-a75_88yp
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-a75_88yp
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 916d8bc82ea43f637047ed989fb7dbc491f56895
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Processing ./pynsee.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
  Using cached pandas-1.5.2-cp310-cp310-

In [213]:
#Importations des bases de données

#Base de communes
df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )

#Base des lycées
dflyc = pd.read_excel('base_def_SansDOMTOM.xlsx')

#Base des coordonnées géographiques des communes 
dfgeo = pd.read_csv('communes-departement-region.csv')

In [214]:
#NettoyageBaseGéo
dfgeo=dfgeo[dfgeo['code_postal']<97000]
dfgeo = dfgeo.drop(["code_postal", "libelle_acheminement", "ligne_5", "nom_commune_complet", "nom_departement", "nom_region","article"], axis=1)
dfgeo

#NettoyageBaseLycCom
dflyc.columns
dflyc = dflyc.drop(["Unnamed: 0", "Valeur ajoutée du taux de réussite_GNLE", "Valeur ajoutée du taux de réussite_Valeur ajoutée techno", "Nombre d'élèves présents au Bac_TOTAL_LGT","Taux de réussite bruts_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_LGT", "Valeur ajoutée du taux de réussite_TOTAL_Pro", "Nbre de ménages fiscaux_NBMEN19","Nbre de personnes dans les ménages fiscaux_NBPERS19", "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19", "coordonnee_X", "coordonnee_Y", "epsg"], axis=1)
dflyc

#Amélioration base communes
df_com.columns = df_com.columns.map('_'.join).str.strip('_')
df_com = df_com.drop(["Nbre de ménages fiscaux_NBMEN19", "Nbre d'unités de consommation dans les ménages fiscaux_NBUC19"], axis=1)
df_com

,Code géographique_CODGEO,Libellé géographique_LIBGEO,Nbre de personnes dans les ménages fiscaux_NBPERS19,Médiane (€)_Q219
0,01001,L'Abergement-Clémenciat,819,24210
1,01002,L'Abergement-de-Varey,267,23380
2,01004,Ambérieu-en-Bugey,15041,19690
3,01005,Ambérieux-en-Dombes,1823,24100
4,01007,Ambronay,2857,23340
...,...,...,...,...
31359,97420,Sainte-Suzanne,23842,14100
31360,97421,Salazie,6936,7740
31361,97422,Le Tampon,78013,12370
31362,97423,Les Trois-Bassins,6989,13060


In [215]:
#Fusion des bases

#Fusion commune/géographie
dfcomgeo = df_com.merge(dfgeo, left_on='Code géographique_CODGEO', right_on = 'code_commune_INSEE')
dfcomgeo = dfcomgeo.drop(["nom_commune_postal", "nom_commune"], axis=1)

#Fusion commune, lycée 
dfmodel = pd.merge(dflyc, dfcomgeo, how = 'outer', left_on = "Code_commune", right_on ='code_commune_INSEE')

In [223]:
dfmodel['Libellé géographique_LIBGEO'] = dfmodel['Libellé géographique_LIBGEO'].str.upper()
dfmodelc = dfmodel.copy()


dfmodelc["Informations établissement_Ville"]= dfmodelc["Informations établissement_Ville"].fillna(0)
dfmodelcnz = dfmodelc[dfmodelc["Informations établissement_Ville"]!=0]
dfmodelcz = dfmodelc[dfmodelc["Informations établissement_Ville"]==0]
for i in range(6800):
    dfmodelcnz.loc[dfmodelcnz.index[i], "Libellé géographique_LIBGEO"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Informations établissement_Ville"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "code_commune_INSEE"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Code_commune"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "Médiane (€)_Q219_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"Médiane (€)_Q219_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "latitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"latitude_x"]
    dfmodelcnz.loc[dfmodelcnz.index[i], "longitude_y"]=dfmodelcnz.loc[dfmodelcnz.index[i],"longitude_x"]
dfmodelc = pd.concat([dfmodelcnz, dfmodelcz])
dfmodelc = dfmodelc.drop(["Code_commune","Informations établissement_Ville","code_commune", "latitude_x", "longitude_x", "Médiane (€)_Q219_x", "Code géographique_CODGEO"], axis=1)
dfmodelc

,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Libellé géographique_LIBGEO,Nbre de personnes dans les ménages fiscaux_NBPERS19,Médiane (€)_Q219_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,BARCELONNETTE,NaN,19070.0,04019,44.388778,6.651860,NaN,NaN
1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,BARCELONNETTE,NaN,19070.0,04019,44.388778,6.651860,NaN,NaN
2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,DIGNE LES BAINS,NaN,19840.0,04070,44.100803,6.234041,NaN,NaN
3,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,DIGNE LES BAINS,NaN,19840.0,04070,44.073766,6.181939,NaN,NaN
4,0040007L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PROFESSIONNEL BEAU DE ROCHAS,PU,NaN,NaN,NaN,NaN,104.0,87.0,DIGNE LES BAINS,NaN,19840.0,04070,44.100068,6.234224,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VILLERON,1570.0,24910.0,95675,49.060843,2.534511,95,11.0
36429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VILLERS-EN-ARTHIES,505.0,27360.0,95676,49.085900,1.730396,95,11.0
36430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VILLIERS-ADAM,809.0,33010.0,95678,49.070289,2.239509,95,11.0
36431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VILLIERS-LE-SEC,221.0,25690.0,95682,49.074309,2.386890,95,11.0


In [ ]:
dfmodelc["Nbre de personnes dans les ménages fiscaux_NBPERS19"] = dfmodelc["Nbre de personnes dans les ménages fiscaux_NBPERS19"].fillna(0)

for i in range(4209):
    if dfmodelc.loc[dfmodel.index[i], "Nbre de personnes dans les ménages fiscaux_NBPERS19"]==0:
        a = dfmodelc.loc[dfmodelc.index[i], 'code_commune_INSEE']
        j=0
        while df_com.loc[dfmodelc.index[j], 'Code géographique_CODGEO']!=a:
            dfmodelc.loc[dfmodelc.index[i],  "Nbre de personnes dans les ménages fiscaux_NBPERS19"] = df_com.loc[dfmodelc.index[j],  "Nbre de personnes dans les ménages fiscaux_NBPERS19"] 
            j = j+1
dfmodelc
                          


In [219]:
dfmodelc

,Identifiant_de_l_etablissement,Informations établissement_Académie,Informations établissement_Département,Informations établissement_Etablissement,Informations établissement_Secteur,Nombre d'élèves présents au Bac_GNLE,Taux de réussite bruts_GNLE,Nombre d'élèves présents au Bac_Somme Techno,Taux de réussite bruts_Taux brut techno,Nombre d'élèves présents au Bac_TOTAL_Pro,Taux de réussite bruts_TOTAL_Pro,Nbre de personnes dans les ménages fiscaux_NBPERS19,Médiane (€)_Q219_y,code_commune_INSEE,latitude_y,longitude_y,code_departement,code_region
0,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.),PU,54.0,98.0,NaN,NaN,NaN,NaN,887.0,19070.0,04019,44.388778,6.651860,NaN,NaN
1,0040003G,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ANDRE HONNORAT (PROFESSIONNEL),PU,NaN,NaN,NaN,NaN,15.0,87.0,887.0,19070.0,04019,44.388778,6.651860,NaN,NaN
2,0040027H,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE ALEXANDRA DAVID NEEL,PU,122.0,99.0,78.0,93.435897,NaN,NaN,763.0,19840.0,04070,44.100803,6.234041,NaN,NaN
3,0040490L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PIERRE-GILLES DE GENNES,PU,98.0,100.0,67.0,98.447761,NaN,NaN,763.0,19840.0,04070,44.073766,6.181939,NaN,NaN
4,0040007L,AIX-MARSEILLE,ALPES DE HTE PROVENCE,LYCEE PROFESSIONNEL BEAU DE ROCHAS,PU,NaN,NaN,NaN,NaN,104.0,87.0,763.0,19840.0,04070,44.100068,6.234224,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1570.0,24910.0,95675,49.060843,2.534511,95,11.0
36429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,505.0,27360.0,95676,49.085900,1.730396,95,11.0
36430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,809.0,33010.0,95678,49.070289,2.239509,95,11.0
36431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.0,25690.0,95682,49.074309,2.386890,95,11.0
